In [ ]:
''' IFC Query
    Run queries in ifc files and visualize the results'''


import ifcopenshell
from ifcopenshell.util.selector import Selector
import ipywidgets as widgets


class Viewer:

    
    def __init__(self, file, elements):
        import k3d
        import ifcopenshell
        import ifcopenshell.geom
        
        self.file = file
        self.elements = elements
        # Set renderer
        self.settings = ifcopenshell.geom.settings()
        self.settings.set(self.settings.USE_WORLD_COORDS, True)
        self.plot = k3d.plot()

    def close(self):
        if self.plot:
            self.plot.close()
            
    def display(self):
        import ifcopenshell
        import ifcopenshell.geom
        import k3d

        for product in self.file.by_type('IfcProduct'):
            if product.is_a('IfcOpeningElement') or product.is_a('IfcSpace'): continue
            if product.Representation :  
                # Change color of selected elements   
                if product in self.elements:
                    opacidade = 1.0
                    cor = 0xdd2779
                else:
                    opacidade = 0.15
                    cor = 0xc0aec 
                # Create shape
                shape = ifcopenshell.geom.create_shape(self.settings, product)
                vertices = shape.geometry.verts
                faces = shape.geometry.faces

                # Add meshes
                self.plot += k3d.mesh(vertices, faces, opacity=opacidade, color=cor)        

        # Render
        self.plot.mode='Change'
        self.plot.camera_mode='orbit'
        self.plot.display()
   
def run(event):
    
    try:
        #File sample : ./modelos_ifc/SampleHouse.ifc
        ifc= ifcopenshell.open(file.value)
        
        selector = Selector()
        elements = selector.parse(ifc, query.value)

        if elements:
 
            v = Viewer(ifc, elements)
            print('Query :', query.value)
            v.display()

        else:
            print("Nenhum elemento encontrado!")
    except:
        print("Não foi possível executar a pesquisa!")

        
# Montagem dos elementos interativos

file=widgets.Text(description='IFC File :')
query=widgets.Text(description='Query :')
button = widgets.Button(
    description='Display',
    tooltip='Click to View',
    button_style='',
)
button.on_click(run)


widgets.HBox([file,query, button])



Error: Session cannot generate requests